In [ ]:
import os
from socket import *

users = []

def login(user,password):
    file = open('users.txt','r')
    global users
    lines = file.read().splitlines()
    file.close()
    if len(lines):
        for line in lines:
            if line != '':
                users.append(line.split('|'))
            
    for i in range(len(users)):
        if [user,password] == users[i]:
            return True
    return False

CHUNKSIZE = 1_000_000

sock = socket()
sock.bind(('',5000))
sock.listen(1)

while True:
    client,address = sock.accept()
    data = client.recv(1024).decode()
    user_name = ((data.split('|'))[0]).strip()
    pass_word = ((data.split('|'))[1]).strip()
    with client:
        if login(user_name, pass_word):
            
            #sent emails
            for path,dirs,files in os.walk(f'{user_name}/sent'):
                for file in files:
                    filename = os.path.join(path,file)
                    relpath = os.path.relpath(filename,'')
                    filesize = os.path.getsize(filename)

                    with open(filename,'rb') as f:
                        client.sendall(relpath.encode() + b'\n')
                        client.sendall(str(filesize).encode() + b'\n')

                        # Send the file in chunks so large files can be handled.
                        while True:
                            data = f.read(CHUNKSIZE)
                            if not data: break
                            client.sendall(data)
                            
            #received emails
            for path,dirs,files in os.walk(f'{user_name}/received'):
                for file in files:
                    filename = os.path.join(path,file)
                    relpath = os.path.relpath(filename,'')
                    filesize = os.path.getsize(filename)

                    with open(filename,'rb') as f:
                        client.sendall(relpath.encode() + b'\n')
                        client.sendall(str(filesize).encode() + b'\n')

                        # Send the file in chunks so large files can be handled.
                        while True:
                            data = f.read(CHUNKSIZE)
                            if not data: break
                            client.sendall(data)
        else:
            client.send('Login Failed'.encode())